In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder,PowerTransformer, StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

import pickle
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
df = pd.read_csv('outlier_removed.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,age,workclass,fnlwgt,education,education-num,marital,occupation,relationship,race,sex,capital,capital.1,hours,country,salary
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,32.5,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            32561 non-null  float64
 1   workclass      32561 non-null  object 
 2   fnlwgt         32561 non-null  float64
 3   education      32561 non-null  object 
 4   education-num  32561 non-null  float64
 5   marital        32561 non-null  object 
 6   occupation     32561 non-null  object 
 7   relationship   32561 non-null  object 
 8   race           32561 non-null  object 
 9   sex            32561 non-null  object 
 10  capital        32561 non-null  float64
 11  capital.1      32561 non-null  float64
 12  hours          32561 non-null  float64
 13  country        32561 non-null  object 
 14  salary         32561 non-null  object 
dtypes: float64(6), object(9)
memory usage: 3.7+ MB


In [6]:
df.isnull().sum()

age              0
workclass        0
fnlwgt           0
education        0
education-num    0
marital          0
occupation       0
relationship     0
race             0
sex              0
capital          0
capital.1        0
hours            0
country          0
salary           0
dtype: int64

In [7]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital,occupation,relationship,race,sex,capital,capital.1,hours,country,salary
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,32.5,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [10]:
df1 = df.copy()

In [13]:
df1.shape

(32561, 15)

In [39]:
X1 = df1.drop(columns=['salary'],axis=1)
y= df1['salary']

In [40]:
X1 = X1.select_dtypes(include=[object])
X1.head(3)

,workclass,education,marital,occupation,relationship,race,sex,country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States


In [41]:
X1.columns

Index(['workclass', 'education', 'marital', 'occupation', 'relationship',
       'race', 'sex', 'country'],
      dtype='object')

In [21]:
from sklearn import preprocessing

In [22]:
le = preprocessing.LabelEncoder()

In [42]:
df1 = X1.apply(le.fit_transform)

In [43]:
df1.head()

,workclass,education,marital,occupation,relationship,race,sex,country
0,7,9,4,1,1,4,1,39
1,6,9,2,4,0,4,1,39
2,4,11,0,6,1,4,1,39
3,4,1,2,6,0,2,1,39
4,4,9,2,10,5,2,0,5


In [49]:
df1.shape

(32561, 8)

In [44]:
df2 = df.select_dtypes(include=np.number)
df2.head()

,age,fnlwgt,education-num,capital,capital.1,hours
0,39.0,77516.0,13.0,0.0,0.0,40.0
1,50.0,83311.0,13.0,0.0,0.0,32.5
2,38.0,215646.0,9.0,0.0,0.0,40.0
3,53.0,234721.0,7.0,0.0,0.0,40.0
4,28.0,338409.0,13.0,0.0,0.0,40.0


In [50]:
df2.shape

(32561, 6)

In [51]:
result = pd.concat([df1, df2], axis=1)

In [52]:
result

,workclass,education,marital,occupation,relationship,race,sex,country,age,fnlwgt,education-num,capital,capital.1,hours
0,7,9,4,1,1,4,1,39,39.0,77516.0,13.0,0.0,0.0,40.0
1,6,9,2,4,0,4,1,39,50.0,83311.0,13.0,0.0,0.0,32.5
2,4,11,0,6,1,4,1,39,38.0,215646.0,9.0,0.0,0.0,40.0
3,4,1,2,6,0,2,1,39,53.0,234721.0,7.0,0.0,0.0,40.0
4,4,9,2,10,5,2,0,5,28.0,338409.0,13.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,4,7,2,13,5,4,0,39,27.0,257302.0,12.0,0.0,0.0,38.0
32557,4,11,2,7,0,4,1,39,40.0,154374.0,9.0,0.0,0.0,40.0
32558,4,11,6,1,4,4,0,39,58.0,151910.0,9.0,0.0,0.0,40.0
32559,4,11,4,1,3,4,1,39,22.0,201490.0,9.0,0.0,0.0,32.5


In [56]:
oe = preprocessing.LabelEncoder()

In [59]:
df_2 = pd.get_dummies(y, drop_first=True)

In [61]:
df_2

,>50K
0,0
1,0
2,0
3,0
4,0
...,...
32556,0
32557,1
32558,0
32559,0


In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(result, df_2)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(24420, 14) (24420, 1) (8141, 14) (8141, 1)


In [68]:
#Using LogisticRegression
lr = LogisticRegression()  
pipe = Pipeline([
    ('lr', lr)
    ])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.7615772018179585

In [69]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[6200    0]
 [1941    0]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      6200
           1       0.00      0.00      0.00      1941

    accuracy                           0.76      8141
   macro avg       0.38      0.50      0.43      8141
weighted avg       0.58      0.76      0.66      8141



In [70]:
#Using Support Vector
svc = SVC()  
pipe = Pipeline([
    ('svc', svc)
    ])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.7615772018179585

In [71]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[6200    0]
 [1941    0]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      6200
           1       0.00      0.00      0.00      1941

    accuracy                           0.76      8141
   macro avg       0.38      0.50      0.43      8141
weighted avg       0.58      0.76      0.66      8141



In [72]:
#Using GaussianNB
gaussian = GaussianNB()
pipe = Pipeline([
    ('nb', gaussian)
    ])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.8006387421692667

In [73]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[5766  434]
 [1189  752]]
              precision    recall  f1-score   support

           0       0.83      0.93      0.88      6200
           1       0.63      0.39      0.48      1941

    accuracy                           0.80      8141
   macro avg       0.73      0.66      0.68      8141
weighted avg       0.78      0.80      0.78      8141



In [74]:
#Using KNN Neighbors
knn = KNeighborsClassifier()  
pipe = Pipeline([
    ('svc', knn)
    ])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.7237440117921631

In [75]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[5492  708]
 [1541  400]]
              precision    recall  f1-score   support

           0       0.78      0.89      0.83      6200
           1       0.36      0.21      0.26      1941

    accuracy                           0.72      8141
   macro avg       0.57      0.55      0.55      8141
weighted avg       0.68      0.72      0.69      8141



In [76]:
#Using Decision Tree
tree = DecisionTreeClassifier()
pipe = Pipeline([
    ('tree', tree)
    ])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.7836875076771895

In [77]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[5324  876]
 [ 885 1056]]
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      6200
           1       0.55      0.54      0.55      1941

    accuracy                           0.78      8141
   macro avg       0.70      0.70      0.70      8141
weighted avg       0.78      0.78      0.78      8141



In [78]:
#Using Random Forest Tree
rf = RandomForestClassifier()
pipe = Pipeline([
    ('rf', rf)
    ])
pipe.fit(X_train.values, y_train.values)
y_pred = pipe.predict(X_test.values)
accuracy_score(y_test, y_pred)

0.8299963149490235

In [79]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[5664  536]
 [ 848 1093]]
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      6200
           1       0.67      0.56      0.61      1941

    accuracy                           0.83      8141
   macro avg       0.77      0.74      0.75      8141
weighted avg       0.82      0.83      0.82      8141

